# Table of Contents
### Step 3: Import libraries and customer data set
### Step 4: Wrangle data
### Step 5: Quality and Consistency Checks
### Step 6: Combine customer data w/rest of prepared data
### Step 8: Export new df

### Step 3: Import libraries and customer data set

In [2]:
# Import libraries 
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [3]:
# turning folder path into string
path = r'/Users/davesmac/Desktop/04-2022- Instacart Basket Analysis'

In [4]:
# Import customer data set
customers = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'customers.csv'), index_col = False)

### Step 4: Wrangle Data

In [5]:
customers.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [6]:
# Rename fam_status to marital_status 
customers.rename(columns = {'fam_status' : 'marital_status'}, inplace = True)

#### I decided not to drop any columns from this data set because all columns could be potentially useful according to key questions in project brief. 

### Step 5: Quality and Consistency Checks

In [7]:
customers.describe()

,user_id,Age,n_dependants,income
count,206209.000000,206209.000000,206209.000000,206209.000000
mean,103105.000000,49.501646,1.499823,94632.852548
std,59527.555167,18.480962,1.118433,42473.786988
min,1.000000,18.000000,0.000000,25903.000000
25%,51553.000000,33.000000,0.000000,59874.000000
50%,103105.000000,49.000000,1.000000,93547.000000
75%,154657.000000,66.000000,3.000000,124244.000000
max,206209.000000,81.000000,3.000000,593901.000000


#### Descriptive stats all look normal- nothing concerning

In [8]:
# Check number of rows and columns in customer data set
customers.shape

(206209, 10)

In [9]:
# Check for mixed types
for col in customers.columns.tolist():
  weird = (customers[[col]].applymap(type) != customers[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (customers[weird]) > 0:
    print (col)

First Name


In [10]:
# Change first name column from mix type to string 
customers['First Name'] = customers['First Name'].astype('str')

In [11]:
# Find missing values
customers.isnull().sum()

user_id           0
First Name        0
Surnam            0
Gender            0
STATE             0
Age               0
date_joined       0
n_dependants      0
marital_status    0
income            0
dtype: int64

#### No missing values found

In [12]:
# Find duplicates
customers_dups = customers[customers.duplicated()]

In [13]:
customers_dups

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,marital_status,income


#### No duplicate records found.

In [14]:
# Check data type for user_id
customers['user_id'].dtype

dtype('int64')

In [15]:
# Change data type for user_id to float64
customers['user_id'] = customers['user_id'].astype('float64')

#### Needed to change the data type for user_id to combine with rest of data

### Step 6: Combine customer data w/rest of prepared data

In [16]:
# Import ords_prods_merge
ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_merged.pkl'))

In [17]:
ords_prods_merge.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,first_order,product_id,add_to_cart_order,reordered,...,_merge,price_label,Busiest_days,busiest_period_of_day,max_order,loyalty_flag,avg_price_per_order,spender_flag,order_frequency,order_frequency_flag
0,2539329.0,1.0,1.0,2.0,8.0,NaN,True,196.0,1.0,0.0,...,both,Mid-range product,Regular busy,Average orders,10.0,new customer,6.367797,Low Spender,20.259259,Non-frequent customer
1,2398795.0,1.0,2.0,3.0,7.0,15.0,False,196.0,1.0,1.0,...,both,Mid-range product,Slowest days,Average orders,10.0,new customer,6.367797,Low Spender,20.259259,Non-frequent customer
2,473747.0,1.0,3.0,3.0,12.0,21.0,False,196.0,1.0,1.0,...,both,Mid-range product,Slowest days,Average orders,10.0,new customer,6.367797,Low Spender,20.259259,Non-frequent customer
3,2254736.0,1.0,4.0,4.0,7.0,29.0,False,196.0,1.0,1.0,...,both,Mid-range product,Slowest days,Average orders,10.0,new customer,6.367797,Low Spender,20.259259,Non-frequent customer
4,431534.0,1.0,5.0,4.0,15.0,28.0,False,196.0,1.0,1.0,...,both,Mid-range product,Slowest days,Average orders,10.0,new customer,6.367797,Low Spender,20.259259,Non-frequent customer


In [18]:
ords_prods_merge = ords_prods_merge.drop(columns = ['_merge'])

In [19]:
ords_prods_all = ords_prods_merge.merge(customers, on = 'user_id', indicator = True) 

### Step 8: Export new df

In [20]:
#export ords_prods_all to prepared data
ords_prods_all.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_all.pkl'))